In [163]:
import csv 
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

# Create CSV Files

In [164]:
def convert_to_mib(memory):
    # Convert memory to MiB if necessary
    if 'GiB' in memory:
        memory_in_gib = float(memory[:-3])
        memory_in_mib = memory_in_gib * 1024
        return f"{memory_in_mib:.2f}MiB"
    elif 'MiB' in memory:
        return memory
    else:
        raise ValueError("Invalid memory format")


def extract_info(file_name):
    cpu_percentages = []
    mem_usages = []

    with open(file_name, 'r') as file:
        for line in file:
            values = line.split()
            
            cpu_percentages.append(float(values[2][:-1]))
            mem_usages.append(float(convert_to_mib(values[3])[:-3]))

    return cpu_percentages, mem_usages


def write_to_csv(cpu_percentages, mem_usages, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['CPU (%)', 'Memory Usage (MB)'])
        for i in range(len(cpu_percentages)):
            writer.writerow([cpu_percentages[i], mem_usages[i]])

In [165]:
def list_txt_files_in_folder(folder_path):
    txt_files = []
    try:
        for file in os.listdir(folder_path):
            if file.endswith(".txt") and os.path.isfile(os.path.join(folder_path, file)):
                txt_files.append(file)
    except Exception as e:
        print(f"Error occurred while listing .txt files: {e}")
    return txt_files

In [166]:
input_folder_path = "./txt/"
output_folder_path = "./output/"

files_in_folder = list_txt_files_in_folder(input_folder_path)

for file_name in files_in_folder:
    print(file_name)
    file_path = input_folder_path + file_name
    output_file_path = "./output/" + file_name.split(".")[0] + ".csv"

    cpu_percentages, mem_usages = extract_info(file_path)
    write_to_csv(cpu_percentages, mem_usages, output_file_path)
    
    print(f'Data has been written to {output_file_path}')

data_centralized.txt
Data has been written to ./output/data_centralized.csv
data_distribute.txt
Data has been written to ./output/data_distribute.csv


# Generate Charts

In [167]:
def list_csv_files_in_folder(folder_path):
    csv_files = []
    try:
        for file in os.listdir(folder_path):
            if file.endswith(".csv") and os.path.isfile(os.path.join(folder_path, file)):
                csv_files.append(file)
    except Exception as e:
        print(f"Error occurred while listing .csv files: {e}")
    return csv_files

In [169]:
csv_files_in_folder = list_csv_files_in_folder(output_folder_path)
chart_folder_path = "./chart/"

for file_name in csv_files_in_folder:
    mem_usages = []

    file_path = output_folder_path + file_name
    print(file_path)

    with open(file_path, "r") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            mem_usage = float(row["Memory Usage (MB)"])
            mem_usages.append(mem_usage)
    
    x = np.arange(len(mem_usages))

    y = np.array(mem_usages)
    f = interp1d(x, y, kind='cubic')
    x_new = np.linspace(x.min(), x.max(), 300)
    y_smooth = f(x_new)
    
    plt.plot(x_new, y_smooth, marker='', linestyle='-')
    plt.title('Memory Usage Over Time')
    plt.xlabel('Data Point')
    plt.ylabel('Memory Usage (MiB)')
    plt.grid(True)
    plt.tight_layout()

    chart_file_name = file_name.split(".")[0] + ".png"
    chart_file_path = chart_folder_path + chart_file_name
    plt.savefig(chart_file_path, dpi=300)
    plt.clf()
    # break

./output/data_distribute.csv
./output/data_centralized.csv


<Figure size 640x480 with 0 Axes>